In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv('Train_aBjfeNk.csv')
test=pd.read_csv('Test_LqhgPWU.csv')
sub=pd.read_csv('sample_submission_wyi0h0z.csv')

In [4]:
train.shape, test.shape

((8068, 11), (2627, 10))

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [6]:
test.shape[0]

2627

In [7]:
(train['ID'].isin(test['ID'])).sum()

2332

In [8]:
(test.ID.isin(train.ID)==False).sum()

295

In [9]:
print('% of test IDs which are already available in train:', 2332/2627)
print('% of test IDs to be predicted:', 295/2627)

% of test IDs which are already available in train: 0.8877046060144652
% of test IDs to be predicted: 0.11229539398553483


In [10]:
#Make two test dataframes, one to be filled from train data and other to be predicted

In [11]:
true_test = test[test.ID.isin(train.ID)==False]
true_test

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
6,459005,Male,Yes,61,Yes,Doctor,5.0,Low,3.0,Cat_6
19,459045,Female,Yes,88,Yes,Lawyer,1.0,Average,4.0,Cat_6
32,459090,Male,No,31,No,Artist,1.0,Low,2.0,Cat_6
38,459116,Male,Yes,60,Yes,Artist,2.0,Average,5.0,Cat_6
43,459121,Female,Yes,51,Yes,Artist,3.0,Average,6.0,Cat_6
...,...,...,...,...,...,...,...,...,...,...
2607,467913,Male,Yes,48,Yes,Artist,3.0,Low,1.0,Cat_6
2612,467930,Male,Yes,45,Yes,Entertainment,4.0,Average,2.0,Cat_6
2616,467938,Male,Yes,46,Yes,Entertainment,5.0,Low,1.0,Cat_2
2624,467960,Female,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6


In [12]:
data=pd.merge(test[['ID']], train[['ID','Segmentation']], on='ID', how ='left')
data

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,A
3,459000,C
4,459001,C
...,...,...
2622,467954,D
2623,467958,A
2624,467960,NaN
2625,467961,B


In [13]:
data[data.Segmentation.isnull()]

,ID,Segmentation
6,459005,NaN
19,459045,NaN
32,459090,NaN
38,459116,NaN
43,459121,NaN
...,...,...
2607,467913,NaN
2612,467930,NaN
2616,467938,NaN
2624,467960,NaN


In [14]:
data=pd.DataFrame.dropna(data)
data

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,A
3,459000,C
4,459001,C
...,...,...
2620,467949,D
2621,467950,D
2622,467954,D
2623,467958,A


In [15]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [16]:
#Label encode the cat_cols in train

In [17]:
list(train.select_dtypes(include='object').columns)

['Gender',
 'Ever_Married',
 'Graduated',
 'Profession',
 'Spending_Score',
 'Var_1',
 'Segmentation']

In [18]:
categorical_columns = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']

In [19]:
for c in categorical_columns:
    mapper={y:x for x,y in enumerate(train[c].unique())}
    train[c]=train[c].map(mapper)
    
    mapper={y:x for x,y in enumerate(test[c].unique())}
    true_test[c]=true_test[c].map(mapper)

In [20]:
train['Segmentation']=train['Segmentation'].map({'A':0, 'B':1, 'C':2, 'D':3})

In [21]:
train.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,0,0,22,0,0,1.0,0,4.0,0,3
1,462643,1,1,38,1,1,NaN,1,3.0,0,0


In [22]:
true_test.head(2)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
6,459005,1,0,61,0,5,5.0,0,3.0,0
19,459045,0,0,88,0,7,1.0,1,4.0,0


In [23]:
X_train=train.drop(['ID', 'Segmentation'], axis=1)
y_train=train['Segmentation']

In [24]:
from xgboost import XGBClassifier
model=XGBClassifier(n_estimators=1000, max_depth=5, learning_rate=0.01)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
X_test=true_test.drop('ID', axis=1)
pred_test=model.predict(X_test)
pred_test

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 3, 1, 3, 0, 0, 0, 1, 3, 1, 3,
       3, 0, 3, 0, 3, 3, 3, 1, 3, 3, 1, 3, 3, 0, 3, 1, 0, 2, 3, 0, 3, 0,
       3, 0, 1, 3, 3, 0, 0, 0, 1, 3, 3, 0, 3, 3, 0, 0, 0, 3, 1, 3, 3, 3,
       3, 3, 0, 0, 0, 1, 3, 0, 3, 3, 3, 3, 0, 3, 1, 3, 0, 3, 2, 3, 3, 3,
       0, 0, 3, 1, 3, 0, 1, 1, 0, 1, 1, 1, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0,
       0, 3, 1, 0, 3, 3, 1, 0, 1, 1, 3, 0, 3, 1, 1, 1, 0, 3, 0, 3, 0, 3,
       3, 1, 3, 0, 0, 1, 3, 0, 1, 3, 0, 0, 1, 1, 3, 0, 1, 1, 3, 3, 3, 3,
       1, 3, 3, 0, 3, 3, 0, 0, 3, 1, 1, 1, 3, 3, 1, 0, 0, 1, 3, 0, 0, 3,
       3, 0, 0, 0, 3, 0, 1, 2, 2, 3, 2, 2, 3, 3, 0, 2, 0, 1, 3, 0, 3, 0,
       0, 1, 3, 0, 1, 3, 3, 3, 3, 3, 0, 3, 1, 3, 1, 3, 3, 0, 3, 1, 3, 1,
       1, 3, 0, 0, 0, 0, 3, 0, 3, 3, 3, 2, 1, 0, 0, 0, 0, 0, 3, 1, 3, 3,
       3, 1, 3, 3, 1, 0, 3, 3, 1, 0, 3, 0, 2, 3, 1, 3, 3, 3, 1, 3, 1, 3,
       0, 0, 3, 3, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 2, 1, 3, 1, 0, 3, 3,
       3, 3, 3, 3, 0, 0, 3, 3, 0], dtype=int64)

In [26]:
true_test['Segmentation']=pred_test

In [27]:
true_test['Segmentation']=true_test['Segmentation'].map({0:'A', 1:'B', 2:'C', 3:'D'})

In [28]:
true_test.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
6,459005,1,0,61,0,5,5.0,0,3.0,0,B
19,459045,0,0,88,0,7,1.0,1,4.0,0,A
32,459090,1,1,31,1,6,1.0,0,2.0,0,A
38,459116,1,0,60,0,6,2.0,1,5.0,0,B
43,459121,0,0,51,0,6,3.0,1,6.0,0,B


In [29]:
test_in_train = test[~test.index.isin(true_test.index)]
test_in_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6
...,...,...,...,...,...,...,...,...,...,...
2620,467949,Male,No,21,No,Healthcare,1.0,Low,4.0,Cat_4
2621,467950,Female,No,35,Yes,Entertainment,1.0,Low,2.0,Cat_6
2622,467954,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6
2623,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6


In [30]:
test_in_train['Segmentation']=data['Segmentation']

In [31]:
test_in_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,B
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,C
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,A
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6,C
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6,C
...,...,...,...,...,...,...,...,...,...,...,...
2620,467949,Male,No,21,No,Healthcare,1.0,Low,4.0,Cat_4,D
2621,467950,Female,No,35,Yes,Entertainment,1.0,Low,2.0,Cat_6,D
2622,467954,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,D
2623,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,A


In [32]:
full_test=pd.concat([test_in_train, true_test])
full_test

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6,B
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6,C
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6,A
3,459000,Male,Yes,59,No,Executive,11.0,High,2.0,Cat_6,C
4,459001,Female,No,19,No,Marketing,NaN,Low,4.0,Cat_6,C
...,...,...,...,...,...,...,...,...,...,...,...
2607,467913,1,0,48,0,6,3.0,0,1.0,0,A
2612,467930,1,0,45,0,8,4.0,1,2.0,0,A
2616,467938,1,0,46,0,8,5.0,0,1.0,5,D
2624,467960,0,1,53,0,8,NaN,0,2.0,0,D


In [33]:
sub1=full_test[['ID', 'Segmentation']]
sub1.head()

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,A
3,459000,C
4,459001,C


In [34]:
sub1.shape

(2627, 2)

In [35]:
sub1['Segmentation'].value_counts()

D    768
A    687
B    606
C    566
Name: Segmentation, dtype: int64

In [36]:
sub1.to_csv('Submission_1.csv', index=False)